## Graph Visualization Final Project for CS-UY 3943

Data is from the [OpenFlights](https://github.com/jpatokal/openflights/tree/master/data) dataset. I loaded airport, airline, and route data from the dataset. Note that the data goes up to 2014 only.

In [ ]:
%pip install pandas
%pip install matplotlib
%pip install graphistry
%pip install python-dotenv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plot
from IPython.display import display
import graphistry

from dotenv import load_dotenv
import os

Load data from the dataset into pandas dataframes and drop unnecessary columns.

In [ ]:
# Load data from openflights/ folder
DIR = 'openflights/'

# Load airlines.dat data into a pandas DataFrame
columns = ["Airline ID", "Name", "Alias", "IATA", "ICAO", "Callsign", "Country", "Active"]
airlines_df = pd.read_csv(DIR + 'airlines.dat', delimiter=',', names=columns)

# Drop Alias, IATA, Callsign columns
airlines_df.drop(["Alias", "IATA", "Callsign"], axis=1, inplace=True)

# Load airports.dat data into a pandas DataFrame
columns = ["Airport ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"]
airports_df = pd.read_csv(DIR + 'airports.dat', delimiter=',', names=columns)

# Drop ICAO, Altitude, Timezone, DST, Tz database time zone, Type, Source columns
airports_df.drop(["ICAO", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"], axis=1, inplace=True)

# Load routes.dat data into a pandas DataFrame
columns = ["Airline", "Airline ID", "Source airport", "Source airport ID", "Destination airport", "Destination airport ID", "Codeshare", "Stops", "Equipment"]
routes_df = pd.read_csv(DIR + 'routes.dat', delimiter=',', names=columns)

# Drop Codeshare and Equipment columns
routes_df.drop(["Codeshare", "Equipment"], axis=1, inplace=True)

# Load countries.dat data into a pandas DataFrame
columns = ["Name", "ISO", "DAFIF"]
countries_df = pd.read_csv(DIR + 'countries.dat', delimiter=',', names=columns)

# Drop DAFIF column
countries_df.drop(["DAFIF"], axis=1, inplace=True)

# Load country_mappings.dat data into a pandas DataFrame
columns = ["Continent", "Code", "Country", "ISO2", "ISO3", "Number"]
country_mappings_df = pd.read_csv(DIR + 'country_mappings.dat', delimiter=',', comment='#', names=columns)

# Drop ISO3, Number columns
country_mappings_df.drop(["ISO3", "Number"], axis=1, inplace=True)

# Define continent colors
continent_colors = {
    'Africa': 'green',
    'Antarctica': 'black',
    'Asia': 'blue',
    'Europe': 'yellow',
    'North America': 'red',
    'Oceania': 'aqua',
    'South America': 'orange'
}

In [ ]:
# Print out the first 5 rows of each DataFrame
display(airlines_df.head())
display(airports_df.head())
display(routes_df.head())
display(countries_df.head())
display(country_mappings_df.head())

Data Exploration

In [ ]:
# Bar graph of the airports with the most routes
def display_top_airports(mode):
    routes = routes_df["Source airport" if mode == "source" else "Destination airport"].value_counts().head(10)

    # Get the names and continent color of the airports
    names = []
    colors = []
    for airport in routes.index:
        airport_entry = airports_df[airports_df["IATA"] == airport]
        country = airport_entry["Country"].values[0]
        names.append(airport_entry["Name"].values[0] + " (" + country + ")")
        country_iso = countries_df[countries_df["Name"] == country]["ISO"].values[0]
        
        continent = country_mappings_df[country_mappings_df["ISO2"] == country_iso]["Continent"].values[0]
        colors.append(continent_colors[continent])

    plot.bar(names, routes, color=colors)
    plot.xlabel("Airports")
    plot.ylabel("Number of {} routes".format("source" if mode == "source" else "destination"))
    plot.title("Airports with the most {} routes".format("source" if mode == "source" else "destination"))

    # Rotate the x-axis labels
    plot.xticks(rotation=90)
    plot.show()

display_top_airports("source")
display_top_airports("destination")

In [ ]:
# Bar graph of the continents ordered by decreasing number of routes
def display_continent_routes(mode):
    # Aggregate routes by airport
    routes = routes_df["Source airport" if mode == "source" else "Destination airport"].value_counts()

    # Add country to route
    routes = routes.reset_index()
    routes.columns = ["IATA", "Count"]
    routes = pd.merge(routes, airports_df[["IATA", "Country"]], on="IATA")
    routes.drop(["IATA"], axis=1, inplace=True)
   
    # Aggregate routes by country
    routes = routes.groupby("Country").sum()

    # Add ISO2 to route
    routes = routes.reset_index()
    routes.columns = ["Country", "Count"]
    routes = pd.merge(routes, countries_df[["Name", "ISO"]], left_on="Country", right_on="Name")
    routes.drop(["Name", "Country"], axis=1, inplace=True)

    # Add continent to route
    routes = routes.reset_index()
    routes = pd.merge(routes, country_mappings_df[["ISO2", "Continent"]], left_on="ISO", right_on="ISO2")

    # Aggregate routes by continent
    routes = routes.groupby("Continent").sum()

    plot.bar(routes.index, routes["Count"], color=[continent_colors[continent] for continent in routes.index])
    plot.xlabel("Continents")
    plot.ylabel("Number of {} routes".format("source" if mode == "source" else "destination"))
    plot.title("Continents by number of {} routes".format("source" if mode == "source" else "destination"))

    # Rotate the x-axis labels
    plot.xticks(rotation=90)
    plot.show()

display_continent_routes("source")
display_continent_routes("destination")

In [ ]:
# Graph number of airports per continent
def display_continent_airports():
    airports = airports_df["Country"].value_counts()

    # Add ISO2 to airport
    airports = airports.reset_index()
    airports.columns = ["Country", "Count"]
    airports = pd.merge(airports, countries_df[["Name", "ISO"]], left_on="Country", right_on="Name")
    airports.drop(["Name", "Country"], axis=1, inplace=True)

    # Add continent to airport
    airports = airports.reset_index()
    airports = pd.merge(airports, country_mappings_df[["ISO2", "Continent"]], left_on="ISO", right_on="ISO2")

    # Aggregate airports by continent
    airports = airports.groupby("Continent").sum()

    plot.bar(airports.index, airports["Count"], color=[continent_colors[continent] for continent in airports.index])
    plot.xlabel("Continents")
    plot.ylabel("Number of airports")
    plot.title("Airports per continent")

    # Rotate the x-axis labels
    plot.xticks(rotation=90)
    plot.show()

display_continent_airports()

In [ ]:
# Graph airlines with the most routes
def display_top_airlines():
    routes = routes_df["Airline ID"].value_counts().head(10)

    # Get the names and colors of the airlines
    names = []
    colors = []
    for airline in routes.index:
        # Note: Airline ID is an integer in routes_df but a string in airlines_df
        airline_entry = airlines_df[airlines_df["Airline ID"] == int(airline)]
        country = airline_entry["Country"].values[0]
        names.append(airline_entry["Name"].values[0] + " (" + country + ")")
        country_iso = countries_df[countries_df["Name"] == country]["ISO"].values[0]
        
        continent = country_mappings_df[country_mappings_df["ISO2"] == country_iso]["Continent"].values[0]
        colors.append(continent_colors[continent])

    plot.bar(names, routes, color=colors)
    plot.xlabel("Airlines")
    plot.ylabel("Number of routes")
    plot.title("Airlines with the most routes")

    # Rotate the x-axis labels
    plot.xticks(rotation=90)
    plot.show()

display_top_airlines()

Graph Visualization

In [ ]:
# Load environment variables
load_dotenv()

# Register Graphistry API key
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", personal_key_id=os.getenv("PERSONAL_KEY_ID"), personal_key_secret=os.getenv("PERSONAL_KEY_SECRET"))

In [ ]:
# Drop airports with no routes
edge_nodes = pd.concat([routes_df['Source airport'], routes_df['Destination airport']]).unique()
filtered_airports_df = airports_df[airports_df['IATA'].isin(edge_nodes)]

# Drop routes that don't map to airports
filtered_routes_df = routes_df[
    routes_df['Source airport'].isin(filtered_airports_df['IATA']) &
    routes_df['Destination airport'].isin(filtered_airports_df['IATA'])
]

# Create annot_airports_df with country and continent
annot_airports_df = pd.merge(filtered_airports_df, countries_df, left_on='Country', right_on='Name')
annot_airports_df = pd.merge(annot_airports_df, country_mappings_df, left_on='ISO', right_on='ISO2')

# Add color using continent_colors
annot_airports_df['Color'] = annot_airports_df['Continent'].apply(lambda x: continent_colors[x])

# Drop Name_y, Country_y, ISO3, Number
annot_airports_df.drop(['Name_y', 'Country_y', 'ISO2'], axis=1, inplace=True)

# Rename Name_x, Country_x, ISO2
annot_airports_df.rename(columns={'Name_x': 'Name', 'Country_x': 'Country', 'ISO2': 'ISO'}, inplace=True)

display(annot_airports_df.head())

In [ ]:
# Default Settings - No weights/colors
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(annot_airports_df, 'IATA')
graph = graph.edges(filtered_routes_df, direct=True)

url = graph.plot(render=False)
url


In [ ]:
# Default Settings with colors for each continent
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(annot_airports_df, 'IATA').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(filtered_routes_df, direct=True)

url = graph.plot(render=False)
url

In [ ]:
# Add weights based on distance squared (NOTE: This takes a while to run)
def get_distance_squared(row):
    source = annot_airports_df[annot_airports_df['IATA'] == row['Source airport']]
    destination = annot_airports_df[annot_airports_df['IATA'] == row['Destination airport']]
    if source.empty or destination.empty:
        return 0
    return (source['Latitude'].values[0] - destination['Latitude'].values[0])**2 + (source['Longitude'].values[0] - destination['Longitude'].values[0])**2

filtered_weighted_routes_df = filtered_routes_df.copy()
filtered_weighted_routes_df['Distance Squared'] = filtered_weighted_routes_df.apply(get_distance_squared, axis=1)

In [ ]:
# Default Settings with weights set based on latitude and longitude distance squared
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(annot_airports_df, 'IATA').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(filtered_weighted_routes_df, direct=True).bind(edge_weight='Distance Squared')

url = graph.plot(render=False)
url

In [ ]:
# Aggregate route data by country
country_routes = filtered_routes_df.copy()

# Merge with annot_airports_df to get source country and continent
country_routes = pd.merge(country_routes, annot_airports_df, left_on='Source airport', right_on='IATA')

# Drop Airport ID, Name, City, IATA, Latitude, Longitude, ISO, Color, Code
country_routes.drop(['Airport ID', 'Name', 'City', 'IATA', 'Latitude', 'Longitude', 'ISO', 'Color', 'Code'], axis=1, inplace=True)
country_routes.rename(columns={'Country': 'Source Country', 'Continent': 'Source Continent'}, inplace=True)

# Merge with annot_airports_df to get destination country and continent
country_routes = pd.merge(country_routes, annot_airports_df, left_on='Destination airport', right_on='IATA')

# Drop Airport ID, Name, City, IATA, Latitude, Longitude, ISO, Color, Code
country_routes.drop(['Airport ID', 'Name', 'City', 'IATA', 'Latitude', 'Longitude', 'ISO', 'Color', 'Code'], axis=1, inplace=True)
country_routes.rename(columns={'Country': 'Destination Country', 'Continent': 'Destination Continent'}, inplace=True)

# Aggregate routes by country
country_routes = country_routes.groupby(['Source Country', 'Source Continent', 'Destination Country', 'Destination Continent']).size().reset_index(name='Count')

display(country_routes.head())

# Drop countries with no routes
edge_nodes = pd.concat([country_routes['Source Country'], country_routes['Destination Country']]).unique()
filtered_countries_df = countries_df[countries_df['Name'].isin(edge_nodes)]

# Add color using continent_colors
filtered_countries_df = pd.merge(filtered_countries_df, country_mappings_df, left_on='ISO', right_on='ISO2')

# Drop Code, ISO2
filtered_countries_df.drop(['Code', 'ISO2'], axis=1, inplace=True)
filtered_countries_df['Color'] = filtered_countries_df['Continent'].apply(lambda x: continent_colors[x])

In [ ]:
# Graph country routes without color
graph = graphistry.bind(source='Source Country', destination='Destination Country')
graph = graph.nodes(filtered_countries_df, 'Name')
graph = graph.edges(country_routes, direct=True)

url = graph.plot(render=False)
url

In [ ]:
# Graph country routes with color
graph = graphistry.bind(source='Source Country', destination='Destination Country')
graph = graph.nodes(filtered_countries_df, 'Name').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(country_routes, direct=True)

url = graph.plot(render=False)
url

In [ ]:
# Aggregate route data by continent
continent_routes = country_routes.copy()
continent_routes = continent_routes.groupby(['Source Continent', 'Destination Continent']).size().reset_index(name='Count')

# Drop continents with no routes
edge_nodes = pd.concat([continent_routes['Source Continent'], continent_routes['Destination Continent']]).unique()

# Drop self loops
continent_routes = continent_routes[continent_routes['Source Continent'] != continent_routes['Destination Continent']]

display(continent_routes)

# Create continent nodes from country_mappings_df
continents = country_mappings_df[['Continent']].drop_duplicates()

In [ ]:
# Graph continent routes with color
graph = graphistry.bind(source='Source Continent', destination='Destination Continent')
graph = graph.nodes(continents, 'Continent').encode_point_color('Continent', categorical_mapping=continent_colors, default_mapping='black')
graph = graph.edges(continent_routes, direct=True)

url = graph.plot(render=False)
url

Graph Visualization - Subsets

In [ ]:
# Find the top 50 airports with the most routes
top_airports = pd.concat([filtered_routes_df['Source airport'], filtered_routes_df['Destination airport']]).value_counts().head(50).index

# Filter the routes to only include the top 50 airports
filtered_top_routes_df = filtered_routes_df[
    filtered_routes_df['Source airport'].isin(top_airports) &
    filtered_routes_df['Destination airport'].isin(top_airports)
]

# Drop everything but top 50 airports
filtered_top_airports_df = annot_airports_df[annot_airports_df['IATA'].isin(top_airports)]

# Graph with color based on continent
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(filtered_top_airports_df, 'IATA').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(filtered_top_routes_df, direct=True)

url = graph.plot(render=False)
url

In [ ]:
# Find all routes with multiple stops
multi_stop_routes = filtered_routes_df[filtered_routes_df['Stops'] > 0]

# Drop airports that don't have multiple stops
multi_stop_airports = pd.concat([multi_stop_routes['Source airport'], multi_stop_routes['Destination airport']]).unique()

filtered_multi_stop_routes_df = multi_stop_routes[
    multi_stop_routes['Source airport'].isin(multi_stop_airports) &
    multi_stop_routes['Destination airport'].isin(multi_stop_airports)
]

# Drop everything but multi stop airports
filtered_multi_stop_airports_df = annot_airports_df[annot_airports_df['IATA'].isin(multi_stop_airports)]

# Graph with color based on continent
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(filtered_multi_stop_airports_df, 'IATA').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(filtered_multi_stop_routes_df, direct=True)

url = graph.plot(render=False)
url

In [ ]:
# Display routes in the Southern Hemisphere
southern_routes = filtered_routes_df[
    filtered_routes_df['Source airport'].isin(annot_airports_df[annot_airports_df['Latitude'] < 0]['IATA']) &
    filtered_routes_df['Destination airport'].isin(annot_airports_df[annot_airports_df['Latitude'] < 0]['IATA'])
]

# Drop airports that don't have routes in southern_routes
southern_airports = pd.concat([southern_routes['Source airport'], southern_routes['Destination airport']]).unique()

# Graph with color based on continent
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(annot_airports_df[annot_airports_df['IATA'].isin(southern_airports)], 'IATA').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(southern_routes, direct=True)

url = graph.plot(render=False)
url

In [ ]:
# Display routes in the Northern Hemisphere
northern_routes = filtered_routes_df[
    filtered_routes_df['Source airport'].isin(annot_airports_df[annot_airports_df['Latitude'] > 0]['IATA']) &
    filtered_routes_df['Destination airport'].isin(annot_airports_df[annot_airports_df['Latitude'] > 0]['IATA'])
]

# Drop airports that don't have routes in southern_routes
northern_airports = pd.concat([northern_routes['Source airport'], northern_routes['Destination airport']]).unique()

# Graph with color based on continent
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(annot_airports_df[annot_airports_df['IATA'].isin(northern_airports)], 'IATA').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(northern_routes, direct=True)

url = graph.plot(render=False)
url

In [ ]:
# Drop routes that don't have an American airline
american_airlines = airlines_df[airlines_df['Country'] == 'United States']

# Note: Airline ID is an integer in american airlines, change it to a string
american_airlines['Airline ID'] = american_airlines['Airline ID'].astype(str)

american_routes = filtered_routes_df[
    filtered_routes_df['Airline ID'].isin(american_airlines['Airline ID'])
]

# Drop airports that don't have routes in american_routes
american_airports = pd.concat([american_routes['Source airport'], american_routes['Destination airport']]).unique()

# Graph with color based on continent
graph = graphistry.bind(source='Source airport', destination='Destination airport')
graph = graph.nodes(annot_airports_df[annot_airports_df['IATA'].isin(american_airports)], 'IATA').encode_point_color('Color', categorical_mapping={
    'green': 'green', 'black': 'black', 'blue': 'blue', 'yellow': 'yellow', 'red': 'red', 'aqua': 'aqua', 'orange': 'orange'}, default_mapping='black')
graph = graph.edges(american_routes, direct=True)

url = graph.plot(render=False)
url

Correlation Matrix

In [ ]:
n = 20

# Create correlation matrix comparing top n airports using the countries connected
top_n_airports = pd.concat([filtered_routes_df['Source airport'], filtered_routes_df['Destination airport']]).value_counts().head(n).index

# Filter the routes to only include the top 20 airports
filtered_top_n_routes_df = filtered_routes_df[
    filtered_routes_df['Source airport'].isin(top_n_airports) &
    filtered_routes_df['Destination airport'].isin(top_n_airports)
]

# Drop everything but top n airports
filtered_top_n_airports_df = annot_airports_df[annot_airports_df['IATA'].isin(top_n_airports)]

# Create a matrix of the countries connected by the top n airports
# Rows = top n airports, columns = countries
correlation_matrix = pd.DataFrame(index=top_n_airports, columns=annot_airports_df['Country'].unique())

# Fill the matrix with 1 if the airport is connected to the country, 0 otherwise
for airport in top_n_airports:
    connected_countries = filtered_top_n_routes_df[filtered_top_n_routes_df['Source airport'] == airport]['Destination airport']
    for country in correlation_matrix.columns:
        correlation_matrix.at[airport, country] = 1 if country in annot_airports_df[annot_airports_df['IATA'].isin(connected_countries)]['Country'].values else 0

# Swap rows and columns
correlation_matrix = correlation_matrix.transpose()

# Find correlation between the top n airports
correlation_matrix = correlation_matrix.astype(int)
correlation_matrix = correlation_matrix.corr()

display(correlation_matrix)